In [ ]:
Problem link- https://www.kaggle.com/c/customerattritionprediction

Solution algorithm- LDA+others

Accuracy- 0.83050

Position- 7/178


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customerattritionprediction/Sample Submission.csv
/kaggle/input/customerattritionprediction/train.csv
/kaggle/input/customerattritionprediction/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-deep')
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['figure.figsize'] = (12, 8)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 400)
import warnings
warnings.filterwarnings('ignore')
import sklearn.base as skb
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import sklearn.utils as sku
import sklearn.linear_model as sklm
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import catboost as cb
import scipy.stats as sstats
import random
seed = 12
np.random.seed(seed)

from datetime import date

import pandas_profiling as pp

In [3]:
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import torch

# # Imports from our package
# from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
# from lightautoml.dataset.roles import DatetimeRole
# from lightautoml.tasks import Task
# from lightautoml.utils.profiler import Profiler

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV

In [5]:
df_train =pd.read_csv("/kaggle/input/customerattritionprediction/train.csv")
df_test = pd.read_csv("/kaggle/input/customerattritionprediction/test.csv")

In [6]:
profile = pp.ProfileReport(df_train, title='Pandas Profiling Report', explorative=True)
profile.to_file("profile.html")

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df_train.head()

,ID,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,SettlementProcess,QuarterlyPayment,GrandPayment,CustomerAttrition
0,B60LHTZLLPFU4IULNPUF,Female,No,No,Yes,48,Yes,Wifi Broadband,No,Yes,Yes,No,Check,96.26,4628.13,Yes
1,1RUJ7ONYZHYTWKOKO1SD,Male,No,Yes,No,45,Yes,Wifi Broadband,No,No,No,Yes,Bank,18.88,922.09,No
2,DZ9EEOQWSN0MYCBUPAM5,Male,Yes,Yes,No,59,Yes,Wifi Broadband,Yes,No,Yes,No,Card,106.02,6205.89,Yes
3,YZ43L1UQUQACTL1PXXZG,Male,No,No,No,14,Yes,Wifi Broadband,No,No,No,No,Check,80.11,1248.66,No
4,6P28152QGF60PDLSCB9M,Female,No,Yes,Yes,25,Yes,Wifi Broadband,No,Yes,No,Yes,Check,51.53,1285.32,No


In [8]:
y_train = df_train['CustomerAttrition']
test_id = df_test['ID']
all_data = pd.concat([df_train, df_test], axis=0, sort=False)

In [9]:
all_data = all_data.drop(['ID'], axis=1)
all_data.head()

,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,SettlementProcess,QuarterlyPayment,GrandPayment,CustomerAttrition
0,Female,No,No,Yes,48,Yes,Wifi Broadband,No,Yes,Yes,No,Check,96.26,4628.13,Yes
1,Male,No,Yes,No,45,Yes,Wifi Broadband,No,No,No,Yes,Bank,18.88,922.09,No
2,Male,Yes,Yes,No,59,Yes,Wifi Broadband,Yes,No,Yes,No,Card,106.02,6205.89,Yes
3,Male,No,No,No,14,Yes,Wifi Broadband,No,No,No,No,Check,80.11,1248.66,No
4,Female,No,Yes,Yes,25,Yes,Wifi Broadband,No,Yes,No,Yes,Check,51.53,1285.32,No


In [10]:
all_data['GrandPayment'].fillna(round(all_data['GrandPayment'].mean()), inplace=True,)

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [12]:
all_data=MultiColumnLabelEncoder(columns = ['sex','4GService','SettlementProcess']).fit_transform(all_data)
all_data.head()

,sex,Aged,Married,TotalDependents,ServiceSpan,MobileService,4GService,CyberProtection,HardwareSupport,TechnicalAssistance,FilmSubscription,SettlementProcess,QuarterlyPayment,GrandPayment,CustomerAttrition
0,0,No,No,Yes,48,Yes,2,No,Yes,Yes,No,2,96.26,4628.13,Yes
1,1,No,Yes,No,45,Yes,2,No,No,No,Yes,0,18.88,922.09,No
2,1,Yes,Yes,No,59,Yes,2,Yes,No,Yes,No,1,106.02,6205.89,Yes
3,1,No,No,No,14,Yes,2,No,No,No,No,2,80.11,1248.66,No
4,0,No,Yes,Yes,25,Yes,2,No,Yes,No,Yes,2,51.53,1285.32,No


In [13]:
all_data = all_data.replace({'Yes': 1, 'No': 0})

In [14]:
train_dataset =all_data[:len(y_train)]
test_tobepredicted = all_data[len(y_train):]

test_1 = test_tobepredicted.drop(['CustomerAttrition'],axis = 1)

train_features = train_dataset.copy()
train_labels = train_features.pop('CustomerAttrition')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train_features,train_labels,test_size=0.30)

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

In [17]:
import xgboost as xgb

XGB = xgb.XGBClassifier(objective="binary:logistic").fit(X_train, y_train)
xgb_accuracy = XGB.score(X_test, y_test)
xgb_accuracy

[06:08:16] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7586750788643533

In [18]:
# from catboost import CatBoostClassifier

# clf = CatBoostClassifier(
#     # iterations=50,
#     random_seed=42,
#     # learning_rate=0.5,
#     custom_loss=['AUC', 'Accuracy']
# )

# clf.fit(
#     X_train, y_train,
#     eval_set=(X_test, y_test),
#     verbose=False,
#     plot=True
# )

In [19]:
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier(verbose=4)
clf_mlp.fit(X_train, y_train)

y_pred_mlp = clf_mlp.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
print(acc_mlp)

Iteration 1, loss = 13.47444009
Iteration 2, loss = 5.79002815
Iteration 3, loss = 2.17813020
Iteration 4, loss = 0.97953839
Iteration 5, loss = 0.60743273
Iteration 6, loss = 0.47360532
Iteration 7, loss = 0.49677880
Iteration 8, loss = 0.52550064
Iteration 9, loss = 0.54671360
Iteration 10, loss = 0.78441218
Iteration 11, loss = 0.63150312
Iteration 12, loss = 0.51293609
Iteration 13, loss = 0.77045117
Iteration 14, loss = 1.01603261
Iteration 15, loss = 0.76256593
Iteration 16, loss = 0.77095583
Iteration 17, loss = 0.58138417
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
0.7760252365930599


In [20]:
gb =  GradientBoostingClassifier(learning_rate=0.01)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(gb, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
gb_fit_time = scores['fit_time'].mean()
gb_score_time = scores['score_time'].mean()
gb_accuracy = scores['test_accuracy'].mean()
gb_precision = scores['test_precision_macro'].mean()
gb_recall = scores['test_recall_macro'].mean()
gb_f1 = scores['test_f1_weighted'].mean()
gb_roc = scores['test_roc_auc'].mean()

gb_accuracy

0.7820003261179733

In [21]:
gb =  GradientBoostingClassifier(learning_rate=0.01)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(gb, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
bc_accuracy = scores['test_accuracy'].mean()
bc_accuracy

0.7820003261179733

In [22]:
LR = LogisticRegression()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LR, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
LR_accuracy = scores['test_accuracy'].mean()

LR_accuracy

0.7932876788851986

In [23]:
decision_tree = DecisionTreeClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(decision_tree, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
dtree_accuracy = scores['test_accuracy'].mean()

dtree_accuracy

0.7256349109290284

In [24]:
SVM = SVC(probability = True)

scoring = ['accuracy','precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(SVM, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
SVM_accuracy = scores['test_accuracy'].mean()

SVM_accuracy

0.7362488192815138

In [25]:
LDA = LinearDiscriminantAnalysis()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LDA, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
LDA_accuracy = scores['test_accuracy'].mean()

LDA_accuracy

0.7982460967755086

In [26]:
QDA = QuadraticDiscriminantAnalysis()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(QDA, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
QDA_accuracy = scores['test_accuracy'].mean()

QDA_accuracy

0.7651078643468722

In [27]:
random_forest = RandomForestClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(random_forest, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
forest_accuracy = scores['test_accuracy'].mean()

forest_accuracy

0.7910303482738658

In [28]:
# from sklearn.model_selection import GridSearchCV

# # # from sklearn.model_selection import GridSearchCV
# # # Create the parameter grid based on the results of random search 
# # param_grid = {
# #     'bootstrap': [True],
# #     'max_depth': [4,5,6,7,8],
# #     'max_features': ['auto',],
# #     'min_samples_leaf': [3, 4, 5],
# #     'min_samples_split': [8, 10, 12],
# #     'n_estimators': [100, 200, 300, 1000]
# # }

# rfc=RandomForestClassifier(random_state=42)
# param_grid = { 
#     'n_estimators': [100,200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [3,4,5,6],
#     'criterion' :['gini', 'entropy'],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10],
# }
# # # Create a based model
# # # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rfc, param_grid = param_grid, 
#                           cv = 10)



In [29]:
# grid_search.fit(train_features, train_labels)

# grid_search.best_params_

In [30]:
# grid_search.best_score_

In [31]:
sm = SVC(gamma='scale')

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(sm, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
sm_accuracy = scores['test_accuracy'].mean()
sm_accuracy

0.7362500509559335

In [32]:
KNN = KNeighborsClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(KNN, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
KNN_accuracy = scores['test_accuracy'].mean()

KNN_accuracy

0.7621601239248298

In [33]:
bayes = GaussianNB()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(bayes, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
bayes_accuracy = scores['test_accuracy'].mean()

bayes_accuracy

0.7491082711670948

In [34]:
# Comapared different classifiers based on their performance on validation accuaracy and used LDA to predict attrition 
# Got rank of 7 on Private leaderboard

In [35]:
LDA = LinearDiscriminantAnalysis()
LDA.fit(train_features,train_labels)
prediction = LDA.predict(test_1)

In [36]:
dic = {1:"Yes", 0:"No"}

prediction = [dic.get(n, n) for n in prediction]

In [37]:
sub = pd.DataFrame()
sub['ID'] = test_id
sub['CustomerAttrition'] = prediction
sub.to_csv('mysubmission.csv',index=False)